<a href="https://colab.research.google.com/github/alonlavian/FaceCrop/blob/master/FaceCrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trying to enlarge my data set using Google vision API

For the following to work you'll have to setup:
*   service account credentials - https://cloud.google.com/docs/authentication/production#obtaining_and_providing_service_account_credentials_manually
*   google billing  - https://support.google.com/googleapi/answer/6158867?hl=en



## Setup

###  Install google cloud vision

In [0]:
!pip install --upgrade google-cloud-vision


### Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

### Add vision application credentilas to env.

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/ML/Misc/vision_service_account.json"

##Code

###Request face annotations from vision

In [0]:
from google.cloud import vision
from google.cloud.vision import types

def detect_face(face_file, max_results=4):
  
    client = vision.ImageAnnotatorClient()
    
    content = face_file.read()
    image = types.Image(content=content)
    
    return client.face_detection(image=image, max_results=max_results).face_annotations

###Draw a border around the detect faces

In [0]:
from PIL import Image

def highlight_faces(image, faces, output_filename):

  im = Image.open(image)
  draw = ImageDraw.Draw(im)
  for face in faces:
      box = [(vertex.x, vertex.y)
             for vertex in face.fd_bounding_poly.vertices]
      draw.line(box + [box[0]], width=10, fill='#00ff00')
  im.save(output_filename)
  

###Crop around the detected faces 

In [0]:
from PIL import Image

def crop_faces(image_path,image_name,cropped_images_lib):
    with open(image_path, 'rb') as image:
        faces = detect_face(image, 8)
      
    im = Image.open(image_path)
    
    for idx,face in enumerate(faces):
      vects = face.fd_bounding_poly.vertices
      im2 = im.crop([vects[0].x, vects[0].y, vects[2].x - 1, vects[2].y - 1]) 
      cropped_image_path = cropped_images_lib + 'face' + str(idx+1) + '_' + image_name
      im2.save(cropped_image_path, 'JPEG')

###Crop one image

In [0]:
image_name = '1.jpg'
image_path = '/1.jpg'
crop_faces(image_path, image_name, '/')

###Crop from a folder and save

In [0]:
original_images_dir = os.fsencode('/content/drive/My Drive/Google Photos/2019')

cropped_images_lib  = '/content/drive/My Drive/CropMix/'


if not os.path.exists(cropped_images_lib):
    os.makedirs(cropped_images_lib)
    
    
for path, dirs, files in os.walk(original_images_dir):
    for file in files:
      image_name = os.fsdecode(file)
      if image_name.endswith(".jpg"):
        image_path = os.path.join(path,file)
        print("Cropping ", image_name)
        crop_faces(image_path,image_name,cropped_images_lib)            